In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
spark_context = SparkContext("local", "first_app")
spark = SparkSession(spark_context)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sql_context = SQLContext(spark_context)

In [2]:
text_file = sql_context.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', quote='"', delimiter=',').load("/home/jovyan/work/GuatemalaExportsTo.csv")
rdd_filter = text_file.rdd.map(tuple)
rdd_countries = rdd_filter.map(lambda word: (word[4], word[1]))
rdd_countries.take(5)

[('China', 18490000.0),
 ('China', 9830000.0),
 ('China', 8360000.0),
 ('China', 7110000.0),
 ('China', 4070000.0)]

In [3]:
rdd_countries_export = rdd_countries.reduceByKey(lambda a,b: a+b)
print(rdd_countries_export.collect())

[('China', 64176766.0), ('Mexico', 511215129.0), ('Canada', 222575942.0), ('Japon', 164579976.0), ('United Kingdom', 101910872.0), ('Costa Rica', 428419775.0), ('Saudi Arabia', 111274535.0), ('Gabon', 6420.0), ('Italy', 166680831.0), ('France', 34023574.0), ('Brazil', 39146101.0), ('Germany', 144422819.0), ('Honduras', 967311616.0), ('Venezuela', 14998293.0), ('El Salvador', 1217138626.0), ('United States', 3803123168.0)]


In [4]:
rdd_countries_export_sort = spark_context.parallelize(rdd_countries_export.sortBy(lambda a: a[1], False).take(5))
print(rdd_countries_export_sort.collect())

[('United States', 3803123168.0), ('El Salvador', 1217138626.0), ('Honduras', 967311616.0), ('Mexico', 511215129.0), ('Costa Rica', 428419775.0)]


In [5]:
countries_name = rdd_countries_export_sort.map(lambda x: x[0])
countries_quantity = rdd_countries_export_sort.map(lambda x: x[1])
print(countries_name.collect())
print(countries_quantity.collect())

['United States', 'El Salvador', 'Honduras', 'Mexico', 'Costa Rica']
[3803123168.0, 1217138626.0, 967311616.0, 511215129.0, 428419775.0]


In [6]:
fig = go.Figure(data=go.Bar(x=countries_name.collect(),y=countries_quantity.collect()))
fig.update_layout(title_text='Top 5 countries with higher exports value',title_font_size=30,
                  yaxis=dict(title='Total',title_font_size=25),
                  xaxis=dict(title='Country',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('report1.html', auto_open=True)